In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
import sys, os
sys.path.append('../../src/')
# import toolbox
from toolbox import *

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [41]:
def get_cross_hedge_h(spot, future):
    # p.81 of Hull
    rho = np.corrcoef(spot, future)[0, 1]
    sigma_s = np.std(spot)
    sigma_f = np.std(future)
    return rho*sigma_s/sigma_f
    

In [42]:
result_names = ['BBT_Tiingo',
                'BBT_future_Tiingo_ada', 
                'BBT_future_Tiingo_eth',
                'BBT_future_Tiingo_ltc',
                'BBT_future_Tiingo_xrp',
                'BBT_future_BITX',
                'BBT_future_BITW20', 
                'BBT_future_BITW70', 
                'BBT_future_BITW100', 
                'BBT_future_CRIX']

coin_names = ['btc', 'ada', 'eth', 'ltc', 'xrp',
              'BITX', 'BITW20', 'BITW70', 'BITW100', 'CRIX']

coin_names_2 = ['bitcoin', 'ada', 'eth', 'ltc', 'xrp',
              'BITX', 'BITW20', 'BITW70', 'BITW100', 'CRIX']


In [104]:
for i in range(len(result_names)):
    test_df = []
    
    print(result_names[i])
    train_path = '../../processed_data/%s/train/'%result_names[i]
    test_path = '../../processed_data/%s/test/'%result_names[i]

    result_path = '../../results/%s/MM/OHR_NIG.csv'%result_names[i]
    OHR = pd.read_csv(result_path)
    OHR = OHR.iloc[:,1:]

    ls = [l for l in os.listdir(train_path) if l.endswith('.csv')]

    np.mean(np.unique(OHR.file) == np.sort(ls)) == 1

    for file in ls:
        train = pd.read_csv(train_path+file)
        test = pd.read_csv(test_path+file)
        rs_train = train.loc[:, 'log return '+coin_names_2[i]]
        rf_train = train.loc[:, 'log return future']
        _h = get_cross_hedge_h(rs_train, rf_train)
        
        
        rs_test = test.loc[:, 'log return '+coin_names_2[i]]
        rf_test = test.loc[:, 'log return future']

        test.loc[:, 'rh'] = rs_test - _h*rf_test
        test.loc[:, 'h'] = _h
        test = test.loc[:, ['Date', 
                    'log return '+coin_names_2[i],
                    'log return future', 'rh', 'h']]
        
        test = test.rename(columns={'log return '+coin_names_2[i]:'rs', 
                     'log return future':'rf'})
        
        test_df.append(test)
        
        
        h_pd = pd.DataFrame({'file': [file],
                             'risk measure':['CH'], 
                             'OHR': [_h], 
                             'copula':['CH']})
        

        OHR = OHR.append(h_pd)
    
    test_df = pd.concat(test_df)
    test_df.Date = pd.to_datetime(test_df.Date)
    test_df = test_df.sort_values('Date', ascending=False)
    test_df.to_csv('../../results/%s/MM/rh_cross_hedge.csv'%result_names[i])
    
    OHR = OHR.reset_index(drop=True) 
    OHR.to_csv(result_path)

BBT_Tiingo
BBT_future_Tiingo_ada
BBT_future_Tiingo_eth
BBT_future_Tiingo_ltc
BBT_future_Tiingo_xrp
BBT_future_BITX
BBT_future_BITW20
BBT_future_BITW70
BBT_future_BITW100
BBT_future_CRIX


In [105]:
test_df

,Date,rs,rf,rh,h
0,2021-05-27 20:00:00+00:00,0.005573,0.006583,-0.000762,0.962404
1,2021-05-26 20:00:00+00:00,0.054012,0.032643,0.022596,0.962404
2,2021-05-25 20:00:00+00:00,-0.057626,-0.062245,0.002279,0.962404
3,2021-05-24 20:00:00+00:00,0.093271,0.094330,0.002487,0.962404
4,2021-05-21 20:00:00+00:00,-0.131176,-0.108809,-0.026459,0.962404
0,2021-05-20 20:00:00+00:00,0.033315,0.021291,0.013029,0.952777
1,2021-05-19 20:00:00+00:00,-0.157294,-0.090465,-0.071100,0.952777
2,2021-05-18 20:00:00+00:00,-0.012067,-0.019694,0.006696,0.952777
3,2021-05-17 20:00:00+00:00,-0.123533,-0.131645,0.001895,0.952777
4,2021-05-14 20:00:00+00:00,0.054760,0.036850,0.019650,0.952777
